<a href="https://colab.research.google.com/github/L1KASA/DL_Course/blob/main/lab_4/6409_%D0%A2%D0%B0%D1%85%D0%BC%D0%B0%D0%B7%D0%BE%D0%B2%D0%B0%D0%90%D0%A8_%D0%9B%D0%A04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 4

6409 Чернышова Дана

In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [2]:
def load_data(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    data = tf.keras.datasets.mnist.load_data()
    (X_train, y_train), (X_test, y_test) = data
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
Train data shape:  (49000, 28, 28)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 28, 28)
Validation labels shape:  (1000,)
Test data shape:  (10000, 28, 28)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


X_train, y_train, X_val, y_val, X_test, y_test = load_data()
X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [5]:
device = '/device:GPU:0'

class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [6]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(channel_1, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        scores = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return scores

In [7]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [8]:
print_every = 200

def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):


        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [9]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.648425579071045, Accuracy: 14.0625, Val Loss: 2.4256842136383057, Val Accuracy: 23.100000381469727
Iteration 200, Epoch 1, Loss: 0.5067327618598938, Accuracy: 85.26119232177734, Val Loss: 0.4295625686645508, Val Accuracy: 86.5999984741211
Iteration 400, Epoch 1, Loss: 0.4059617519378662, Accuracy: 88.1935806274414, Val Loss: 0.3539341390132904, Val Accuracy: 89.5
Iteration 600, Epoch 1, Loss: 0.361200213432312, Accuracy: 89.45767211914062, Val Loss: 0.33000025153160095, Val Accuracy: 90.5


Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [10]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, 0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.2932896614074707, Accuracy: 14.0625, Val Loss: 2.2945306301116943, Val Accuracy: 10.300000190734863
Iteration 200, Epoch 1, Loss: 0.5406829118728638, Accuracy: 84.375, Val Loss: 0.3682888150215149, Val Accuracy: 88.5
Iteration 400, Epoch 1, Loss: 0.3615179657936096, Accuracy: 89.42877197265625, Val Loss: 0.20971842110157013, Val Accuracy: 93.0999984741211
Iteration 600, Epoch 1, Loss: 0.2863776981830597, Accuracy: 91.63373565673828, Val Loss: 0.18891380727291107, Val Accuracy: 93.4000015258789


# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [11]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (28, 28, 1)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.2397053241729736, Accuracy: 7.8125, Val Loss: 2.634352445602417, Val Accuracy: 20.80000114440918
Iteration 200, Epoch 1, Loss: 0.5109735131263733, Accuracy: 84.82587432861328, Val Loss: 0.4397605061531067, Val Accuracy: 86.4000015258789
Iteration 400, Epoch 1, Loss: 0.41054168343544006, Accuracy: 87.87406921386719, Val Loss: 0.3604540228843689, Val Accuracy: 89.80000305175781
Iteration 600, Epoch 1, Loss: 0.36423879861831665, Accuracy: 89.24188232421875, Val Loss: 0.3280705511569977, Val Accuracy: 90.80000305175781


Альтернативный менее гибкий способ обучения:

In [12]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.2155 - sparse_categorical_accuracy: 0.9375


[0.21547406911849976, 0.9375]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [13]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(28, (5, 5), padding='same', activation='relu', kernel_initializer=tf.initializers.VarianceScaling(scale=2.0)),
        tf.keras.layers.Conv2D(14, (3, 3), padding='same', activation='relu', kernel_initializer=tf.initializers.VarianceScaling(scale=2.0)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax', kernel_initializer=tf.initializers.VarianceScaling(scale=2.0))
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.723869562149048, Accuracy: 17.1875, Val Loss: 2.7944717407226562, Val Accuracy: 8.399999618530273
Iteration 200, Epoch 1, Loss: 0.5216559171676636, Accuracy: 84.09515380859375, Val Loss: 0.47859349846839905, Val Accuracy: 84.5
Iteration 400, Epoch 1, Loss: 0.40278005599975586, Accuracy: 87.83120727539062, Val Loss: 0.3334046006202698, Val Accuracy: 89.4000015258789
Iteration 600, Epoch 1, Loss: 0.3461413085460663, Accuracy: 89.6370620727539, Val Loss: 0.2627509832382202, Val Accuracy: 91.0999984741211


In [14]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.1002 - sparse_categorical_accuracy: 0.9711


[0.10023532062768936, 0.9710999727249146]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [15]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [16]:
input_shape = (28, 28, 1)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.8709096908569336, Accuracy: 7.8125, Val Loss: 2.3923752307891846, Val Accuracy: 23.299999237060547
Iteration 200, Epoch 1, Loss: 0.5085484981536865, Accuracy: 84.9657974243164, Val Loss: 0.4503844976425171, Val Accuracy: 86.0
Iteration 400, Epoch 1, Loss: 0.40910598635673523, Accuracy: 88.01824188232422, Val Loss: 0.3686492443084717, Val Accuracy: 89.0999984741211
Iteration 600, Epoch 1, Loss: 0.3620755076408386, Accuracy: 89.43167877197266, Val Loss: 0.3304932713508606, Val Accuracy: 90.9000015258789


Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [17]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Определение сверточных слоев с активацией ReLU
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))

        # Определение слоев полносвязной части сети
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Применение сверточных слоев
        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.max_pool(x)

        # Применение полносвязных слоев
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x



print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.316258430480957, Accuracy: 15.625, Val Loss: 2.795074939727783, Val Accuracy: 16.0
Iteration 700, Epoch 1, Loss: 0.13723257184028625, Accuracy: 95.92546081542969, Val Loss: 0.07823485136032104, Val Accuracy: 97.89999389648438
Iteration 1400, Epoch 2, Loss: 0.037943799048662186, Accuracy: 98.7844467163086, Val Loss: 0.05978196859359741, Val Accuracy: 98.29999542236328
Iteration 2100, Epoch 3, Loss: 0.019766999408602715, Accuracy: 99.3739013671875, Val Loss: 0.06128118187189102, Val Accuracy: 98.0999984741211
Iteration 2800, Epoch 4, Loss: 0.012801360338926315, Accuracy: 99.54336547851562, Val Loss: 0.06432227045297623, Val Accuracy: 97.69999694824219
Iteration 3500, Epoch 5, Loss: 0.009357052855193615, Accuracy: 99.6925048828125, Val Loss: 0.0683261826634407, Val Accuracy: 98.5
Iteration 4200, Epoch 6, Loss: 0.009005882777273655, Accuracy: 99.70939636230469, Val Loss: 0.07362035661935806, Val Accuracy: 98.0999984741211
Iteration 4900, Epoch 7, Loss: 0.01038

In [18]:
def model_init_fn():
    return CustomConvNet()

# Let's try SGD optimizer
def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.SGD(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.3291828632354736, Accuracy: 1.5625, Val Loss: 2.328279733657837, Val Accuracy: 6.5
Iteration 700, Epoch 1, Loss: 1.234477162361145, Accuracy: 69.9246597290039, Val Loss: 0.5403664708137512, Val Accuracy: 84.69999694824219
Iteration 1400, Epoch 2, Loss: 0.36387741565704346, Accuracy: 89.781005859375, Val Loss: 0.3965161144733429, Val Accuracy: 88.20000457763672
Iteration 2100, Epoch 3, Loss: 0.2915494441986084, Accuracy: 91.65751647949219, Val Loss: 0.3530992865562439, Val Accuracy: 89.4000015258789
Iteration 2800, Epoch 4, Loss: 0.25652801990509033, Accuracy: 92.59132385253906, Val Loss: 0.31251928210258484, Val Accuracy: 90.4000015258789
Iteration 3500, Epoch 5, Loss: 0.22536197304725647, Accuracy: 93.64273834228516, Val Loss: 0.2882891297340393, Val Accuracy: 91.5
Iteration 4200, Epoch 6, Loss: 0.20305775105953217, Accuracy: 94.24696350097656, Val Loss: 0.26293718814849854, Val Accuracy: 91.69999694824219
Iteration 4900, Epoch 7, Loss: 0.1853310912847519

In [19]:
class CustomConvNet_drp(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet_drp, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Определение сверточных слоев
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same')
        self.conv2 = tf.keras.layers.Conv2D(16, (3, 3), padding='same')

        # Определение функций активации и слоев dropout
        self.relu1 = tf.keras.layers.ReLU()
        self.dropout1 = tf.keras.layers.Dropout(0.5)
        self.relu2 = tf.keras.layers.ReLU()
        self.dropout2 = tf.keras.layers.Dropout(0.5)

        # Определение слоев полносвязной части сети
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Применение сверточных слоев
        x = self.conv1(input_tensor)
        x = self.relu1(x)
        x = self.dropout1(x, training=training)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.dropout2(x, training=training)

        # Применение полносвязных слоев
        x = self.flatten(x)
        x = self.dense(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


In [20]:
def model_init_fn():
    return CustomConvNet_drp()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.370997905731201, Accuracy: 10.9375, Val Loss: 2.271935224533081, Val Accuracy: 15.000000953674316
Iteration 700, Epoch 1, Loss: 0.25752124190330505, Accuracy: 92.1875, Val Loss: 0.15156270563602448, Val Accuracy: 94.5
Iteration 1400, Epoch 2, Loss: 0.09926659613847733, Accuracy: 96.96849822998047, Val Loss: 0.09858410805463791, Val Accuracy: 96.80000305175781
Iteration 2100, Epoch 3, Loss: 0.07536294311285019, Accuracy: 97.60820007324219, Val Loss: 0.09660657495260239, Val Accuracy: 96.9000015258789
Iteration 2800, Epoch 4, Loss: 0.0649985522031784, Accuracy: 97.9373779296875, Val Loss: 0.0729275494813919, Val Accuracy: 97.69999694824219
Iteration 3500, Epoch 5, Loss: 0.056527961045503616, Accuracy: 98.2015151977539, Val Loss: 0.09612524509429932, Val Accuracy: 97.29999542236328
Iteration 4200, Epoch 6, Loss: 0.049447447061538696, Accuracy: 98.43328857421875, Val Loss: 0.07374364882707596, Val Accuracy: 97.79999542236328
Iteration 4900, Epoch 7, Loss: 0.04

In [21]:
def model_init_fn():
    return CustomConvNet_drp()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.SGD(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.3291735649108887, Accuracy: 10.9375, Val Loss: 2.3251702785491943, Val Accuracy: 10.199999809265137
Iteration 700, Epoch 1, Loss: 1.2164877653121948, Accuracy: 60.485023498535156, Val Loss: 0.561538815498352, Val Accuracy: 82.9000015258789
Iteration 1400, Epoch 2, Loss: 0.5384864211082458, Accuracy: 83.75984191894531, Val Loss: 0.4854775369167328, Val Accuracy: 86.0
Iteration 2100, Epoch 3, Loss: 0.47686639428138733, Accuracy: 85.68486785888672, Val Loss: 0.46093156933784485, Val Accuracy: 86.5999984741211
Iteration 2800, Epoch 4, Loss: 0.4537081718444824, Accuracy: 86.5432357788086, Val Loss: 0.4372051954269409, Val Accuracy: 87.5999984741211
Iteration 3500, Epoch 5, Loss: 0.42267048358917236, Accuracy: 87.57865905761719, Val Loss: 0.4211602210998535, Val Accuracy: 89.20000457763672
Iteration 4200, Epoch 6, Loss: 0.4020264446735382, Accuracy: 88.22017669677734, Val Loss: 0.4122369587421417, Val Accuracy: 88.70000457763672
Iteration 4900, Epoch 7, Loss: 0.

Опишите все эксперименты, результаты. Сделайте выводы.

Основные выводы по проделанной работе:
1. Лучше всего данные работают с моделью без Dropout слоев как с Adam, так и с SGD. Но даже так все эксперименты показали отличные результаты.
2. В нашем случае наиболее эффективным оптимизатором оказался Adam.